In [2]:
import pandas as pd
import numpy as np

import pytz
import astral as ast
from astral import LocationInfo
from astral.sun import sun
from datetime import datetime, timedelta, date

In [3]:
# read in with pandas
winter1 = pd.read_csv ('jfk20-21data.txt')
winter2 = pd.read_csv ('jfk21-22data.txt')
winter3 = pd.read_csv ('jfk22-23data.txt')

In [4]:
# rename columns
winter1 = winter1.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter2 = winter2.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})
winter3 = winter3.rename(columns = {'valid': 'datetime', 'p01m': 'precip(mm)'})

# datetime format
winter1['datetime'] = pd.to_datetime(winter1['datetime'])
winter2['datetime'] = pd.to_datetime(winter2['datetime'])
winter3['datetime'] = pd.to_datetime(winter3['datetime'])

In [5]:
# mask out days without precipitation
codes = ['GR','GS','IC','PL','PY','RA','SG','SH','SN','SQ','TS','UP']

mask1 = winter1['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask2 = winter2['wxcodes'].apply(lambda x: any(sub in x for sub in codes))
mask3 = winter3['wxcodes'].apply(lambda x: any(sub in x for sub in codes))

precip_days1 = winter1[mask1]
precip_days2 = winter2[mask2]
precip_days3 = winter3[mask3]

In [6]:
precip_days2

,station,datetime,precip(mm),wxcodes
28,JFK,2021-12-02 04:51:00,T,-RA
29,JFK,2021-12-02 05:51:00,0.51,-RA
33,JFK,2021-12-02 08:40:00,T,-RA
34,JFK,2021-12-02 08:51:00,T,-RA
129,JFK,2021-12-06 05:51:00,0.25,-RA
...,...,...,...,...
2412,JFK,2022-02-25 13:59:00,0.51,-FZRA BR
2413,JFK,2022-02-25 14:11:00,1.78,FZRA BR
2414,JFK,2022-02-25 14:37:00,3.56,FZRA BR
2415,JFK,2022-02-25 14:51:00,5.08,-FZRA BR


In [7]:
# replace Trace with 0.0001
precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
precip_days3['precip(mm)'] = precip_days3['precip(mm)'].replace('T', 0.0001)

C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\3060002225.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = precip_days1['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\3060002225.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = precip_days2['precip(mm)'].replace('T', 0.0001)
C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\3060002225.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of

In [8]:
precip_days3[430:440]

,station,datetime,precip(mm),wxcodes


In [9]:
# make sure numeric form
precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
precip_days3['precip(mm)'] = pd.to_numeric(precip_days3['precip(mm)'])

C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\2630444437.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days1['precip(mm)'] = pd.to_numeric(precip_days1['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\2630444437.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['precip(mm)'] = pd.to_numeric(precip_days2['precip(mm)'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\2630444437.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

In [10]:
# eliminate special weather 
precip_days1 = precip_days1[precip_days1['datetime'].dt.minute == 51]
precip_days2 = precip_days2[precip_days2['datetime'].dt.minute == 51]
precip_days3 = precip_days3[precip_days3['datetime'].dt.minute == 51]

In [11]:
precip_days1.dtypes

station               object
datetime      datetime64[ns]
precip(mm)           float64
wxcodes               object
dtype: object

In [12]:
precip_days1['date'] = precip_days1['datetime'].dt.date

In [13]:
jfk = LocationInfo(
    'New York' 'New York', 
    latitude= 40.6446,
    longitude= -73.7797, 
    timezone='America/New_York'
)

jfk

LocationInfo(name='New YorkNew York', region='England', timezone='America/New_York', latitude=40.6446, longitude=-73.7797)

In [14]:
# create a date range (winter 1)
date_range1 = pd.date_range(start='2020-12-01', end='2021-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times1 = []
sunset_times1 = []

# Calculate sunrise and sunset for each date
for date in date_range1:
    s = sun(jfk.observer, date=date)
    
    sunrise_times1.append(s['sunrise'])
    sunset_times1.append(s['sunset'])

# df with results
sun_times1 = pd.DataFrame({
    'datetime': date_range1.date,
    'sunrise': sunrise_times1,
    'sunset': sunset_times1
})

sun_times1['sunrise'] = pd.to_datetime(sun_times1['sunrise'])
sun_times1['sunset'] = pd.to_datetime(sun_times1['sunset'])

In [15]:
# create a date range (winter 2)
date_range2 = pd.date_range(start='2021-12-01', end='2022-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times2 = []
sunset_times2 = []

# Calculate sunrise and sunset for each date
for date in date_range2:
    s = sun(jfk.observer, date=date)
    
    sunrise_times2.append(s['sunrise'])
    sunset_times2.append(s['sunset'])

# df with results
sun_times2 = pd.DataFrame({
    'datetime': date_range2.date,
    'sunrise': sunrise_times2,
    'sunset': sunset_times2
})

sun_times2['sunrise'] = pd.to_datetime(sun_times2['sunrise'])
sun_times2['sunset'] = pd.to_datetime(sun_times2['sunset'])

In [16]:
# create a date range (winter 3)
date_range3 = pd.date_range(start='2022-12-01', end='2023-2-28', tz='America/New_York')

# lists to store sunrise and sunset times
sunrise_times3 = []
sunset_times3 = []

# Calculate sunrise and sunset for each date
for date in date_range3:
    s = sun(jfk.observer, date=date)
    
    sunrise_times3.append(s['sunrise'])
    sunset_times3.append(s['sunset'])

# df with results
sun_times3 = pd.DataFrame({
    'datetime': date_range3.date,
    'sunrise': sunrise_times3,
    'sunset': sunset_times3
})

sun_times3['sunrise'] = pd.to_datetime(sun_times3['sunrise'])
sun_times3['sunset'] = pd.to_datetime(sun_times3['sunset'])

In [17]:
# add sunrise and sunset times (winter1)
sun_times1['sunrise'] = sun_times1['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times1['sunset'] = sun_times1['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times1['date'] = sun_times1['datetime']
sun_times1 = sun_times1.drop(columns=['datetime'])

print(sun_times1)

                sunrise               sunset        date
0   2020-12-01 07:00:22  2020-12-01 16:28:11  2020-12-01
1   2020-12-02 07:01:22  2020-12-02 16:27:58  2020-12-02
2   2020-12-03 07:02:21  2020-12-03 16:27:48  2020-12-03
3   2020-12-04 07:03:19  2020-12-04 16:27:40  2020-12-04
4   2020-12-05 07:04:16  2020-12-05 16:27:34  2020-12-05
..                  ...                  ...         ...
85  2021-02-24 06:36:17  2021-02-24 17:40:48  2021-02-24
86  2021-02-25 06:34:49  2021-02-25 17:41:57  2021-02-25
87  2021-02-26 06:33:19  2021-02-26 17:43:06  2021-02-26
88  2021-02-27 06:31:50  2021-02-27 17:44:15  2021-02-27
89  2021-02-28 06:30:19  2021-02-28 17:45:23  2021-02-28

[90 rows x 3 columns]


In [18]:
# add sunrise and sunset times (winter2)

sun_times2['sunrise'] = sun_times2['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times2['sunset'] = sun_times2['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times2['date'] = sun_times2['datetime']
sun_times2 = sun_times2.drop(columns=['datetime'])

print(sun_times2)

                sunrise               sunset        date
0   2021-12-01 07:00:07  2021-12-01 16:28:14  2021-12-01
1   2021-12-02 07:01:07  2021-12-02 16:28:01  2021-12-02
2   2021-12-03 07:02:07  2021-12-03 16:27:50  2021-12-03
3   2021-12-04 07:03:05  2021-12-04 16:27:41  2021-12-04
4   2021-12-05 07:04:02  2021-12-05 16:27:34  2021-12-05
..                  ...                  ...         ...
85  2022-02-24 06:36:38  2022-02-24 17:40:31  2022-02-24
86  2022-02-25 06:35:10  2022-02-25 17:41:41  2022-02-25
87  2022-02-26 06:33:41  2022-02-26 17:42:50  2022-02-26
88  2022-02-27 06:32:11  2022-02-27 17:43:58  2022-02-27
89  2022-02-28 06:30:41  2022-02-28 17:45:07  2022-02-28

[90 rows x 3 columns]


In [19]:
# add sunrise and sunset times (winter3)

sun_times3['sunrise'] = sun_times3['sunrise'].dt.strftime('%Y-%m-%d %H:%M:%S')
sun_times3['sunset'] = sun_times3['sunset'].dt.strftime('%Y-%m-%d %H:%M:%S')

sun_times3['date'] = sun_times3['datetime']
sun_times3 = sun_times3.drop(columns=['datetime'])

print(sun_times3)

                sunrise               sunset        date
0   2022-12-01 06:59:52  2022-12-01 16:28:18  2022-12-01
1   2022-12-02 07:00:53  2022-12-02 16:28:04  2022-12-02
2   2022-12-03 07:01:52  2022-12-03 16:27:52  2022-12-03
3   2022-12-04 07:02:51  2022-12-04 16:27:43  2022-12-04
4   2022-12-05 07:03:48  2022-12-05 16:27:36  2022-12-05
..                  ...                  ...         ...
85  2023-02-24 06:36:59  2023-02-24 17:40:14  2023-02-24
86  2023-02-25 06:35:31  2023-02-25 17:41:24  2023-02-25
87  2023-02-26 06:34:03  2023-02-26 17:42:33  2023-02-26
88  2023-02-27 06:32:33  2023-02-27 17:43:42  2023-02-27
89  2023-02-28 06:31:03  2023-02-28 17:44:50  2023-02-28

[90 rows x 3 columns]


In [20]:
# add 'date' column to all dataframes
precip_days1['date'] = precip_days1['datetime'].dt.date
precip_days2['date'] = precip_days2['datetime'].dt.date
precip_days3['date'] = precip_days3['datetime'].dt.date

C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\762931898.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  precip_days2['date'] = precip_days2['datetime'].dt.date


In [21]:
# merge precip days and sunrise/sunset times on date 
merged1 = pd.merge(precip_days1, sun_times1, on='date')

merged1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JFK,2020-12-01 01:51:00,0.0001,-RA,2020-12-01,2020-12-01 07:00:22,2020-12-01 16:28:11
1,JFK,2020-12-04 19:51:00,0.0001,-RA,2020-12-04,2020-12-04 07:03:19,2020-12-04 16:27:40
2,JFK,2020-12-04 20:51:00,0.2500,-RA,2020-12-04,2020-12-04 07:03:19,2020-12-04 16:27:40
3,JFK,2020-12-04 21:51:00,0.2500,-RA,2020-12-04,2020-12-04 07:03:19,2020-12-04 16:27:40
4,JFK,2020-12-04 22:51:00,0.0001,-RA,2020-12-04,2020-12-04 07:03:19,2020-12-04 16:27:40
...,...,...,...,...,...,...,...
247,JFK,2021-02-27 12:51:00,1.0200,-RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15
248,JFK,2021-02-27 13:51:00,1.2700,-RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15
249,JFK,2021-02-27 14:51:00,2.0300,RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15
250,JFK,2021-02-27 15:51:00,2.5400,-RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15


In [22]:
# merge precip days and sunrise/sunset times on date 
merged2 = pd.merge(precip_days2, sun_times2, on='date')

merged2

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JFK,2021-12-02 04:51:00,0.0001,-RA,2021-12-02,2021-12-02 07:01:07,2021-12-02 16:28:01
1,JFK,2021-12-02 05:51:00,0.5100,-RA,2021-12-02,2021-12-02 07:01:07,2021-12-02 16:28:01
2,JFK,2021-12-02 08:51:00,0.0001,-RA,2021-12-02,2021-12-02 07:01:07,2021-12-02 16:28:01
3,JFK,2021-12-06 05:51:00,0.2500,-RA,2021-12-06,2021-12-06 07:04:58,2021-12-06 16:27:30
4,JFK,2021-12-06 08:51:00,6.3500,RA BR,2021-12-06,2021-12-06 07:04:58,2021-12-06 16:27:30
...,...,...,...,...,...,...,...
237,JFK,2022-02-25 10:51:00,0.7600,-RA,2022-02-25,2022-02-25 06:35:10,2022-02-25 17:41:41
238,JFK,2022-02-25 11:51:00,0.5100,-RA BR,2022-02-25,2022-02-25 06:35:10,2022-02-25 17:41:41
239,JFK,2022-02-25 12:51:00,1.0200,RA BR,2022-02-25,2022-02-25 06:35:10,2022-02-25 17:41:41
240,JFK,2022-02-25 13:51:00,2.7900,-RA BR,2022-02-25,2022-02-25 06:35:10,2022-02-25 17:41:41


In [23]:
# merge precip days and sunrise/sunset times on date 
merged3 = pd.merge(precip_days3, sun_times3, on='date')

merged3

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JFK,2022-12-03 12:51:00,0.0000,-RA BR,2022-12-03,2022-12-03 07:01:52,2022-12-03 16:27:52
1,JFK,2022-12-03 13:51:00,0.5100,-RA BR,2022-12-03,2022-12-03 07:01:52,2022-12-03 16:27:52
2,JFK,2022-12-03 14:51:00,0.2500,-RA BR,2022-12-03,2022-12-03 07:01:52,2022-12-03 16:27:52
3,JFK,2022-12-03 15:51:00,0.7600,-RA BR,2022-12-03,2022-12-03 07:01:52,2022-12-03 16:27:52
4,JFK,2022-12-03 16:51:00,0.5100,-RA BR,2022-12-03,2022-12-03 07:01:52,2022-12-03 16:27:52
...,...,...,...,...,...,...,...
217,JFK,2023-02-22 21:51:00,2.5400,-RA BR,2023-02-22,2023-02-22 06:39:52,2023-02-22 17:37:55
218,JFK,2023-02-23 08:51:00,0.2500,-RA BR,2023-02-23,2023-02-23 06:38:26,2023-02-23 17:39:05
219,JFK,2023-02-23 10:51:00,0.0001,-RA,2023-02-23,2023-02-23 06:38:26,2023-02-23 17:39:05
220,JFK,2023-02-25 15:51:00,0.0001,-SN,2023-02-25,2023-02-25 06:35:31,2023-02-25 17:41:24


In [24]:
# filter out only times after sunrise & before sunset 

days1 = merged1[(merged1['datetime'] >= merged1['sunrise']) & (merged1['datetime'] <= merged1['sunset'])]
days2 = merged2[(merged2['datetime'] >= merged2['sunrise']) & (merged2['datetime'] <= merged2['sunset'])]
days3 = merged3[(merged3['datetime'] >= merged3['sunrise']) & (merged3['datetime'] <= merged3['sunset'])]


days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
11,JFK,2020-12-05 07:51:00,1.78,-RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34
12,JFK,2020-12-05 08:51:00,2.29,RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34
13,JFK,2020-12-05 09:51:00,4.57,+RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34
14,JFK,2020-12-05 10:51:00,3.56,RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34
15,JFK,2020-12-05 11:51:00,5.84,RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34
...,...,...,...,...,...,...,...
247,JFK,2021-02-27 12:51:00,1.02,-RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15
248,JFK,2021-02-27 13:51:00,1.27,-RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15
249,JFK,2021-02-27 14:51:00,2.03,RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15
250,JFK,2021-02-27 15:51:00,2.54,-RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15


In [25]:
# filter out transition hours
days1['sunrise'] = pd.to_datetime(days1['sunrise'])
days1['sunset'] = pd.to_datetime(days1['sunset'])

days1['sunrise+1'] = days1['sunrise'] + timedelta(hours = 1)
days1['sunset-1'] = days1['sunset'] - timedelta(hours = 1)

days1 = days1[days1['datetime'] <= days1['sunset-1']]
days1 = days1[days1['datetime'] >= days1['sunrise+1']]



days2['sunrise'] = pd.to_datetime(days2['sunrise'])
days2['sunset'] = pd.to_datetime(days2['sunset'])

days2['sunrise+1'] = days2['sunrise'] + timedelta(hours = 1)
days2['sunset-1'] = days2['sunset'] - timedelta(hours = 1)

days2 = days2[days2['datetime'] <= days2['sunset-1']]
days2 = days2[days2['datetime'] >= days2['sunrise+1']]



days3['sunrise'] = pd.to_datetime(days3['sunrise'])
days3['sunset'] = pd.to_datetime(days3['sunset'])

days3['sunrise+1'] = days3['sunrise'] + timedelta(hours = 1)
days3['sunset-1'] = days3['sunset'] - timedelta(hours = 1)

days3 = days3[days3['datetime'] <= days3['sunset-1']]
days3 = days3[days3['datetime'] >= days3['sunrise+1']]

C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\2882998222.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunrise'] = pd.to_datetime(days1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\2882998222.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  days1['sunset'] = pd.to_datetime(days1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\2882998222.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexe

In [26]:
days1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset,sunrise+1,sunset-1
12,JFK,2020-12-05 08:51:00,2.29,RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34,2020-12-05 08:04:16,2020-12-05 15:27:34
13,JFK,2020-12-05 09:51:00,4.57,+RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34,2020-12-05 08:04:16,2020-12-05 15:27:34
14,JFK,2020-12-05 10:51:00,3.56,RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34,2020-12-05 08:04:16,2020-12-05 15:27:34
15,JFK,2020-12-05 11:51:00,5.84,RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34,2020-12-05 08:04:16,2020-12-05 15:27:34
16,JFK,2020-12-05 12:51:00,2.03,RA BR,2020-12-05,2020-12-05 07:04:16,2020-12-05 16:27:34,2020-12-05 08:04:16,2020-12-05 15:27:34
...,...,...,...,...,...,...,...,...,...
246,JFK,2021-02-27 11:51:00,1.27,-RA,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15,2021-02-27 07:31:50,2021-02-27 16:44:15
247,JFK,2021-02-27 12:51:00,1.02,-RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15,2021-02-27 07:31:50,2021-02-27 16:44:15
248,JFK,2021-02-27 13:51:00,1.27,-RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15,2021-02-27 07:31:50,2021-02-27 16:44:15
249,JFK,2021-02-27 14:51:00,2.03,RA BR,2021-02-27,2021-02-27 06:31:50,2021-02-27 17:44:15,2021-02-27 07:31:50,2021-02-27 16:44:15


In [27]:
# find total precip over each year
day_total1 = days1['precip(mm)'].sum()
day_total2 = days2['precip(mm)'].sum()
day_total3 = days3['precip(mm)'].sum()

# total number of obs each year
day_rows1 = days1.shape[0]
day_rows2 = days2.shape[0]
day_rows3 = days3.shape[0]

In [28]:
day_precip_per_hour1 = day_total1/ day_rows1
day_precip_per_hour2 = day_total2/ day_rows2
day_precip_per_hour3 = day_total3/ day_rows3


In [29]:
print('2020-2021:', day_precip_per_hour1)
print('2021-2022:', day_precip_per_hour2)
print('2022-2023:', day_precip_per_hour3)

2020-2021: 1.1167307692307693
2021-2022: 0.8669436170212768
2022-2023: 0.507225


In [30]:
# filter out only times after sunset & before NEXT sunrise 

nights1 = merged1[(merged1['datetime'] < merged1['sunrise']) | (merged1['datetime'] > merged1['sunset'])]
nights2 = merged2[(merged2['datetime'] < merged2['sunrise']) | (merged2['datetime'] > merged2['sunset'])]
nights3 = merged3[(merged3['datetime'] < merged3['sunrise']) | (merged3['datetime'] > merged3['sunset'])]

nights1

,station,datetime,precip(mm),wxcodes,date,sunrise,sunset
0,JFK,2020-12-01 01:51:00,0.0001,-RA,2020-12-01,2020-12-01 07:00:22,2020-12-01 16:28:11
1,JFK,2020-12-04 19:51:00,0.0001,-RA,2020-12-04,2020-12-04 07:03:19,2020-12-04 16:27:40
2,JFK,2020-12-04 20:51:00,0.2500,-RA,2020-12-04,2020-12-04 07:03:19,2020-12-04 16:27:40
3,JFK,2020-12-04 21:51:00,0.2500,-RA,2020-12-04,2020-12-04 07:03:19,2020-12-04 16:27:40
4,JFK,2020-12-04 22:51:00,0.0001,-RA,2020-12-04,2020-12-04 07:03:19,2020-12-04 16:27:40
...,...,...,...,...,...,...,...
236,JFK,2021-02-22 18:51:00,3.5600,RA BR,2021-02-22,2021-02-22 06:39:11,2021-02-22 17:38:29
237,JFK,2021-02-22 19:51:00,4.5700,RA BR,2021-02-22,2021-02-22 06:39:11,2021-02-22 17:38:29
238,JFK,2021-02-22 20:51:00,1.0200,-RA BR,2021-02-22,2021-02-22 06:39:11,2021-02-22 17:38:29
239,JFK,2021-02-22 21:51:00,1.2700,-RA BR,2021-02-22,2021-02-22 06:39:11,2021-02-22 17:38:29


In [31]:
# filter out transition hours
nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
nights1['sunset'] = pd.to_datetime(nights1['sunset'])

nights1['sunrise-1'] = nights1['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights1['sunset+1'] = nights1['sunset'] + timedelta(hours = 1)

nights1 = nights1[nights1['datetime'] >= nights1['sunset+1']]
nights1 = nights1[nights1['datetime'] <= nights1['sunrise-1']]



nights2['sunrise'] = pd.to_datetime(nights2['sunrise'])
nights2['sunset'] = pd.to_datetime(nights2['sunset'])

nights2['sunrise-1'] = nights2['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights2['sunset+1'] = nights2['sunset'] + timedelta(hours = 1)

nights2 = nights2[nights2['datetime'] >= nights2['sunset+1']]
nights2 = nights2[nights2['datetime'] <= nights2['sunrise-1']]



nights3['sunrise'] = pd.to_datetime(nights3['sunrise'])
nights3['sunset'] = pd.to_datetime(nights3['sunset'])

nights3['sunrise-1'] = nights3['sunrise'] - timedelta(hours = 1) + timedelta(days = 1)
nights3['sunset+1'] = nights3['sunset'] + timedelta(hours = 1)

nights3 = nights3[nights3['datetime'] >= nights3['sunset+1']]
nights3 = nights3[nights3['datetime'] <= nights3['sunrise-1']]


C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\36922422.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunrise'] = pd.to_datetime(nights1['sunrise'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\36922422.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nights1['sunset'] = pd.to_datetime(nights1['sunset'])
C:\Users\alyha\AppData\Local\Temp\ipykernel_27556\36922422.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_inde

In [32]:
# find total precip over each year
night_total1 = nights1['precip(mm)'].sum()
night_total2 = nights2['precip(mm)'].sum()
night_total3 = nights3['precip(mm)'].sum()

# total number of obs each year
night_rows1 = nights1.shape[0]
night_rows2 = nights2.shape[0]
night_rows3 = nights3.shape[0]

In [33]:
night_precip_per_hour1 = night_total1/ night_rows1
night_precip_per_hour2 = night_total2/ night_rows2
night_precip_per_hour3 = night_total3/ night_rows3

In [34]:
print('2020-2021:', night_precip_per_hour1)
print('2021-2022:', night_precip_per_hour2)
print('2022-2023:', night_precip_per_hour3)

2020-2021: 0.5373867647058825
2021-2022: 0.33635217391304345
2022-2023: 0.7616972222222222


In [35]:
data = pd.concat([days1, nights1, days2, nights2, days3, nights3])

data['date'] = pd.to_datetime(data['date'])
data['month'] = data['date'].dt.month

data['state'] = 'NY'

data.to_csv('jfk.csv', index=False)